In [ ]:
import pathlib
from typing import Dict, List, Union

import matplotlib.pylab as plt
from bokeh.plotting import figure, output_notebook, show

from PIL import Image
from ipywidgets import interact, interactive, fixed, IntSlider, Dropdown

from scheeg import data

# Data

> Data processing/pre-processing functionality

The path to the MATLAB data file is assembled

In [ ]:
input_dir = pathlib.Path.cwd() / 'preprocessed_data'
input_file = input_dir / 'CON_MAT.mat'
assert input_file.exists()
input_file

*MATLAB* file is loaded

In [ ]:
metrics, (n_subjects, n_channels, _, n_frequency_bands, n_samples) = data.read_matlab(input_file)
print(f'{n_subjects=}, {n_channels=}, {n_frequency_bands=}, {n_samples=}')

Every array in the `metrics` dictionary has the same dimensions, given by

In [ ]:
assert metrics['GPDC_H'].shape == metrics['GPDC_S'].shape == metrics['dDTF_H'].shape == metrics['dDTF_S'].shape
metrics['GPDC_H'].shape

*GDPC* is between $0$ and $1$

In [ ]:
metrics['GPDC_H'].min(), metrics['GPDC_H'].max()

and so is *dDTF*

In [ ]:
metrics['dDTF_H'].min(), metrics['dDTF_H'].max()

Diagonal of each sample (for each subject, sample...) is **always** $0$

In [ ]:
metrics['GPDC_H'].diagonal(axis1=1, axis2=2).sum(), metrics['dDTF_H'].diagonal(axis1=1, axis2=2).sum()

Data can be visualized with *matplotlib*. Notice, however, that these are really one-channel images and hence not the nice-(pseudo)colored pictures shown.

In [ ]:
@interact(
    healthy_or_ill=Dropdown(options=[('Healthy', '_H'), ('Ill', '_S')], value='_H', description='Kind'),
    i_subject=IntSlider(description='# subject', min=0, max=n_subjects-1, value=0),
    i_frequency=IntSlider(description='freq. band', min=0, max=n_frequency_bands-1, value=0),
    i_sample=IntSlider(description='# sample', min=0, max=n_samples-1, value=0))
def plot_sample(healthy_or_ill: str, i_subject:int = 0, i_frequency: int = 0, i_sample: int = 0):
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(12,12)
    
    # GDPC
    ax1.imshow(metrics['GPDC' + healthy_or_ill][i_subject, :, :, i_frequency, i_sample])
    ax1.set_title('GDPC')
    
    # dDTF
    ax2.imshow(metrics['dDTF' + healthy_or_ill][i_subject, :, :, i_frequency, i_sample])
    ax2.set_title('dDTF')

Let's collect the first sample of the first frequency for the first healthy and ill subjects.

In [ ]:
sample_healthy = metrics['GPDC_H'][0, :, :, 0, 0]
sample_ill = metrics['GPDC_S'][0, :, :, 0, 0]

Using *pillow* one can turn the array into a grey-scale image.

In [ ]:
im = Image.fromarray(sample_healthy, mode='L')
im

## Merging frequency bands

Every color channel (R,G,B) is obtained from some frequency bands

In [ ]:
rgb_from_freq = {
    'R': range(12), # encompassing approximately delta, theta and alpha bands
    'G': range(12,30), # encompassing approximately the beta band
    'B': range(30,50) # encompassing approximately the gamma band
}

Data in each *range* is averaged. The results for <font color='red'>R</font>, <font color='green'>G</font>, and <font color='blue'>B</font> are stacked back together in a single array. Notice the dimension for the *color channel* in the latter is the 4th one. Also, the old *frequency dimension* is now gone.

In [ ]:
rgb_data = {k: data.average_frequencies(metrics[k], rgb_from_freq.values()) for k in ['GPDC_H', 'GPDC_S']}

The size of any of the resulting arrays is

In [ ]:
next(iter(rgb_data.values())).shape

## Arrays to images

The numbers in the arrays (the metrics *GPDC* and *dDTF*) are between $0$ and $1$, but a color channel should be an **integer** number between $0$ and $255$. Hence, in order to turn the data values into colors we multiply by $255$ and cast the numbers as 8-bits integers.

In [ ]:
scaled_rgb_data = {k: data.to_rgb(v) for k, v in rgb_data.items()}

We can visualize the newly constructed images

In [ ]:
@interact(
    healthy_or_ill=Dropdown(options=[('Healthy', '_H'), ('Ill', '_S')], value='_H', description='Kind'),
    i_subject=IntSlider(description='# subject', min=0, max=n_subjects-1, value=0),
    i_sample=IntSlider(description='# sample', min=0, max=n_samples-1, value=0))
def plot_sample(healthy_or_ill: str, i_subject:int = 0, i_sample: int = 0):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(8,8)
    
    # GDPC
    ax.imshow(scaled_rgb_data['GPDC' + healthy_or_ill][i_subject, ..., i_sample])
    ax.set_title('GDPC')